## 1. Cargar los datos

#### 1.1. Importar librerias y módulos

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import funciones as fun
import variables_nombres as vn
from warnings import simplefilter

In [2]:
#print(dir(fun))
simplefilter( action = "ignore", category = pd.errors.PerformanceWarning )
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.max_columns', 10)

#### 1.2. Importar bases 

Se importan las cuatro bases de manera conjunta, y se agrupan en la lista "bases". Para accerder a cada una de las bases se llama a la lista de bases + su respectivo índice. Ejemplo: bases[0]

In [3]:
pwd

'C:\\Users\\dell\\Documents\\QLAB\\Corruption_Paper\\code\\preprocess_data'

In [4]:
path = r'..\..\output\data_build\*.dta'

In [5]:
bases = fun.importar_bases( path )
for i, base in enumerate( bases ):
    print( i )

0
1
2
3


#### 1.3. Filtrar solo aquellas variables que hicieron match

Se filtra las variables que previamente, en el preprocesamiento en Stata, hicieron match en la realización de merge.

In [ ]:
for i, base in enumerate( bases ):
    bases[i] = bases[i].loc[ bases[i]['_merge'] == 3 ]

#### 1.4. Última limpieza a las bases

Últimas modificaciones a las bases de datos

In [7]:
for i, base in enumerate( bases ):
    data_cols = [ col for col in bases[i].columns if col not in vn.no_predictoras_variables and col not in vn.string_variables ]
    encode_variables = { "sexo":{ "Mujer":1, "Hombre":2 }}
    bases[i] = bases[i][data_cols]
    bases[i] = bases[i].replace( encode_variables )
    bases[i] = bases[i].astype( str ).replace( '', np.nan, regex = True )
    bases[i] = bases[i].astype( str ).replace( '\.+$', np.nan, regex = True ).astype( float )

#### 1.5. Duplicar las bases de datos finales

In [11]:
dfs = [ pd.DataFrame() for x in range( 4 ) ]
for base in bases:
    dfs = bases.copy()

#### 1.6. Asignar nombres a las bases

In [17]:
dfs[0].name = 'base0'
dfs[1].name = 'base1'
dfs[2].name = 'base2'
dfs[3].name = 'base3'

#### 1.7. Determinar el número de variables por cada fuente

In [13]:
fun.contar_variables( dfs )

base0: Variables de Renamu: 693; Variables de SIAF: 4690
base1: Variables de Renamu: 693; Variables de SIAF: 4690
base2: Variables de Renamu: 693; Variables de SIAF: 4690
base3: Variables de Renamu: 693; Variables de SIAF: 4690


#### 1.8. Determinar las dimensiones de cada base

In [14]:
fun.determinar_dimensiones( dfs )

base0 (652, 5393)
base1 (845, 5393)
base2 (1290, 5393)
base3 (1969, 5393)


## 2. Primera forma: imputación con 0

#### 2.1. Imputación

Se imputa los valores perdidos de las variables de SIAF con 0. No se generan variables dummy de control.

In [15]:
fun.imputar_i( dfs, vn.siaf_variables, 0, dummy = False )

[     VFI  sexo  facebook  web  propie_muni  locales_propios  mue_pro  \
 0    1.0   1.0       0.0  1.0          1.0              NaN      1.0   
 1    1.0   1.0       0.0  0.0          1.0              NaN      1.0   
 2    1.0   2.0       0.0  0.0          1.0              NaN      1.0   
 3    NaN   NaN       NaN  NaN          NaN              NaN      NaN   
 4    1.0   2.0       1.0  0.0          0.0              NaN      1.0   
 ..   ...   ...       ...  ...          ...              ...      ...   
 647  1.0   1.0       1.0  1.0          1.0              NaN      1.0   
 648  1.0   2.0       1.0  1.0          1.0              3.0      1.0   
 649  1.0   2.0       0.0  0.0          1.0              1.0      1.0   
 650  1.0   2.0       1.0  1.0          0.0              NaN      1.0   
 651  1.0   1.0       1.0  1.0          1.0              NaN      1.0   
 
      mue_pro_1  mue_pro_2  mue_pro_3  ...  monto_auditado  monto_examinado  \
 0          1.0        0.0        1.0  ... 

#### 2.2. Filtrar por valores perdidos

Se descartan las variables con un porcentaje de valores perdidos mayor o igual al umbral 0.5

In [16]:
fun.filtro_missings( dfs, 0.5 )

[     VFI  sexo  facebook  web  propie_muni  mue_pro  mue_pro_1  mue_pro_2  \
 0    1.0   1.0       0.0  1.0          1.0      1.0        1.0        0.0   
 1    1.0   1.0       0.0  0.0          1.0      1.0        1.0        1.0   
 2    1.0   2.0       0.0  0.0          1.0      1.0        1.0        3.0   
 3    NaN   NaN       NaN  NaN          NaN      NaN        NaN        NaN   
 4    1.0   2.0       1.0  0.0          0.0      1.0        1.0        1.0   
 ..   ...   ...       ...  ...          ...      ...        ...        ...   
 647  1.0   1.0       1.0  1.0          1.0      1.0        1.0        4.0   
 648  1.0   2.0       1.0  1.0          1.0      1.0        1.0        3.0   
 649  1.0   2.0       0.0  0.0          1.0      1.0        1.0        1.0   
 650  1.0   2.0       1.0  1.0          0.0      1.0        1.0        3.0   
 651  1.0   1.0       1.0  1.0          1.0      1.0        1.0        1.0   
 
      mue_pro_3  mue_pro_4  ...  monto_auditado  monto_examina

Se verifica el numero de variables por fuente de cada base luego del paso 2.2.

In [18]:
fun.contar_variables( dfs )

base0: Variables de Renamu: 361; Variables de SIAF: 4690
base1: Variables de Renamu: 371; Variables de SIAF: 4690
base2: Variables de Renamu: 456; Variables de SIAF: 4690
base3: Variables de Renamu: 498; Variables de SIAF: 4690


Se verifica la dimensionalidad de las bases de datos

In [19]:
fun.determinar_dimensiones( dfs )

base0 (652, 5061)
base1 (845, 5066)
base2 (1290, 5156)
base3 (1969, 5194)


Se verifica que las variables pertenecientes a SIAF ya no tengan valores perdidos

In [20]:
df0_siaf = fun.listar_variables( dfs[0], siaf = True )
df1_siaf = fun.listar_variables( dfs[1], siaf = True )
df2_siaf = fun.listar_variables( dfs[2], siaf = True )
df3_siaf = fun.listar_variables( dfs[3], siaf = True )

In [21]:
print( dfs[0][df0_siaf].isnull().sum().sum(),
       dfs[1][df1_siaf].isnull().sum().sum(),
       dfs[2][df2_siaf].isnull().sum().sum(),
       dfs[3][df3_siaf].isnull().sum().sum(), sep = "\n" )

0
0
0
0


#### 2.3. Filtro por variabilidad

Se descartan aquellas variables constantes, es decir, con una variabilidad de 0

In [22]:
fun.filtro_variabilidad( dfs, 0 )

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets[i].drop( columnas, axis = 1, inplace = True )
C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets[i].drop( columnas, axis = 1, inplace = True )
C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

[     sexo  facebook  web  propie_muni  mue_pro_1  mue_pro_2  mue_pro_3  \
 0     1.0       0.0  1.0          1.0        1.0        0.0        1.0   
 1     1.0       0.0  0.0          1.0        1.0        1.0        0.0   
 2     2.0       0.0  0.0          1.0        1.0        3.0        0.0   
 3     NaN       NaN  NaN          NaN        NaN        NaN        NaN   
 4     2.0       1.0  0.0          0.0        1.0        1.0        1.0   
 ..    ...       ...  ...          ...        ...        ...        ...   
 647   1.0       1.0  1.0          1.0        1.0        4.0        4.0   
 648   2.0       1.0  1.0          1.0        1.0        3.0        3.0   
 649   2.0       0.0  0.0          1.0        1.0        1.0        1.0   
 650   2.0       1.0  1.0          0.0        1.0        3.0        0.0   
 651   1.0       1.0  1.0          1.0        1.0        1.0        0.0   
 
      mue_pro_4  mue_pro_5  mue_pro_6  ...  monto_auditado  monto_examinado  \
 0          2.0    

Se verifica la dimensionalidad de las bases tras el paso 2.3.

In [23]:
fun.determinar_dimensiones( dfs )

base0 (652, 3967)
base1 (845, 3972)
base2 (1290, 4165)
base3 (1969, 4201)


#### 2.4. Filtro por correlación

Se realiza un filtro de correlación para cada una de las variables dependientes de la base0. El umbral es 0.8. Para mayor detalle en el funcionamiento del filtro, véase su respectiva sección en el módulo "funciones.py". Posteriormente se agrupa las bases filtradas de la base 0 en la lista "dfs_0_i". Luego, se asigna el nombre a cada una de las bases. Finalmente, se determina las dimensiones de las bases

In [30]:
dfs_0_ci = fun.filtro_correlacion( dfs[0], dfs[0][ 'corrup_intensa' ], 0.8 )
dfs_0_ca = fun.filtro_correlacion( dfs[0], dfs[0][ 'corrup_amplia' ], 0.8 )
dfs_0_mci = fun.filtro_correlacion( dfs[0], dfs[0][ 'monto_corrup1' ], 0.8 )
dfs_0_mca = fun.filtro_correlacion( dfs[0], dfs[0][ 'monto_corrup2' ], 0.8 )
dfs_0_pci = fun.filtro_correlacion( dfs[0], dfs[0][ 'per_corrup1' ], 0.8 )
dfs_0_pca = fun.filtro_correlacion( dfs[0], dfs[0][ 'per_corrup2' ], 0.8 )

In [33]:
filtradas_0 = [ dfs_0_ci, dfs_0_ca, dfs_0_mci, dfs_0_mca, dfs_0_pci, dfs_0_pca ]
dfs_0_i = []
for i, filtrada_0 in enumerate( filtradas_0 ):
    dfs_0_i.append( filtrada_0 )

In [37]:
dfs_0_i[0].name = 'dfs_0_i_ci'
dfs_0_i[1].name = 'dfs_0_i_ca'
dfs_0_i[2].name = 'dfs_0_i_mci'
dfs_0_i[3].name = 'dfs_0_i_mca'
dfs_0_i[4].name = 'dfs_0_i_pci'
dfs_0_i[5].name = 'dfs_0_i_pca'

In [38]:
fun.determinar_dimensiones( dfs_0_i )

dfs_0_i_ci (652, 1266)
dfs_0_i_ca (652, 1271)
dfs_0_i_mci (652, 1277)
dfs_0_i_mca (652, 1270)
dfs_0_i_pci (652, 1268)
dfs_0_i_pca (652, 1258)


Se realiza el mismo procedimiento para cada una de las variables dependientes de la base1.

In [ ]:
dfs_1_ci = fun.filtro_correlacion( dfs[1], dfs[1][ 'corrup_intensa' ], 0.8 )
dfs_1_ca = fun.filtro_correlacion( dfs[1], dfs[1][ 'corrup_amplia' ], 0.8 )
dfs_1_m = fun.filtro_correlacion( dfs[1], dfs[1][ 'monto' ], 0.8 )

In [ ]:
filtradas_1 = [ dfs_1_ci, dfs_1_ca, dfs_1_m ]
dfs_1_i = []
for i, filtrada_1 in enumerate( filtradas_1 ):
    dfs_1_i.append( filtrada_1 )

In [ ]:
dfs_1_i[0].name = 'dfs_1_i_ci'
dfs_1_i[1].name = 'dfs_1_i_ca'
dfs_1_i[2].name = 'dfs_1_i_m'

In [ ]:
fun.determinar_dimensiones( dfs_1_i )

Se realiza el mismo procedimiento para cada una de las bases dependientes de la base2.

In [ ]:
dfs_2_ci = fun.filtro_correlacion( dfs[2], dfs[2][ 'corrup_intensa' ], 0.8 )
dfs_2_ca = fun.filtro_correlacion( dfs[2], dfs[2][ 'corrup_amplia' ], 0.8 )
dfs_2_m = fun.filtro_correlacion( dfs[2], dfs[2][ 'monto' ], 0.8 )

In [ ]:
filtradas_2 = [ dfs_2_ci, dfs_2_ca, dfs_2_m ]
dfs_2_i = []
for i, filtrada_2 in enumerate( filtradas_2 ):
    dfs_2_i.append( filtrada_2 )

In [ ]:
dfs_2_i[0].name = 'dfs_2_i_ci'
dfs_2_i[1].name = 'dfs_2_i_ca'
dfs_2_i[2].name = 'dfs_2_i_m'

Se realiza el mismo procedimiento para cada una de las bases dependientes de la base3.

In [ ]:
dfs_3_ci = fun.filtro_correlacion( dfs[3], dfs[3][ 'corrup_intensa' ], 0.8 )
dfs_3_ca = fun.filtro_correlacion( dfs[3], dfs[3][ 'corrup_amplia' ], 0.8 )
dfs_3_m = fun.filtro_correlacion( dfs[3], dfs[3][ 'monto' ], 0.8 )

In [ ]:
filtradas_3 = [ dfs_3_ci, dfs_3_ca, dfs_3_m ]
dfs_3_i = []
for i, filtrada_3 in enumerate( filtradas_3 ):
    dfs_3_i.append( filtrada_3 )

In [ ]:
dfs_3_i[0].name = 'dfs_3_i_ci'
dfs_3_i[1].name = 'dfs_3_i_ca'
dfs_3_i[2].name = 'dfs_3_i_m'

#### 2.5. Imputar outliers

Se imputa los valores del percentil 1% superior con los valores del percentil 99% para cada una de las variables de SIAF. En este caso, la imputación se realiza sobre las tres bases resultantes de la base0.

In [39]:
fun.imputar_outliers( dfs_0_i, vn.siaf_variables, 0.01 )

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.loc[ dataframe[var] > perc, var ] = perc


[     sexo  facebook  web  mue_pro_1  mue_pro_4  mue_pro_5  mue_pro_6  \
 0     1.0       0.0  1.0        1.0        2.0        0.0        0.0   
 1     1.0       0.0  0.0        1.0        1.0        1.0        0.0   
 2     2.0       0.0  0.0        1.0        2.0        0.0        0.0   
 3     NaN       NaN  NaN        NaN        NaN        NaN        NaN   
 4     2.0       1.0  0.0        1.0        1.0        2.0        0.0   
 ..    ...       ...  ...        ...        ...        ...        ...   
 647   1.0       1.0  1.0        1.0        1.0        6.0        1.0   
 648   2.0       1.0  1.0        1.0        1.0        3.0        0.0   
 649   2.0       0.0  0.0        1.0        1.0        1.0        0.0   
 650   2.0       1.0  1.0        1.0        1.0        6.0        0.0   
 651   1.0       1.0  1.0        1.0        0.0        NaN        NaN   
 
      mue_pro_7  mue_pro_10  mue_pro_13  ...  tdvgtotfun_f5viv  \
 0          2.0         2.0         2.0  ...          0.

Se aplica el mismo procedimiento para las tres bases resultantes de la base1.

In [ ]:
fun.imputar_outliers( dfs_1_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base2.

In [ ]:
fun.imputar_outliers( dfs_2_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base3.

In [ ]:
fun.imputar_outliers( dfs_3_i, vn.siaf_variables, 0.01 )

#### 2.6. Transformaciones logarítmicas

Se realiza una transformación logarítmica a todas las variables pertenecientes a SIAF. La transformación se aplica sobre las tres bases de datos resultantes de la base0.

In [40]:
fun.transformacion_log( dfs_0_i )

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets[i][var] = np.log( datasets[i][var] + 1 )
C:\Anaconda\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Anaconda\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[     sexo  facebook  web  mue_pro_1  mue_pro_4  mue_pro_5  mue_pro_6  \
 0     1.0       0.0  1.0        1.0        2.0        0.0        0.0   
 1     1.0       0.0  0.0        1.0        1.0        1.0        0.0   
 2     2.0       0.0  0.0        1.0        2.0        0.0        0.0   
 3     NaN       NaN  NaN        NaN        NaN        NaN        NaN   
 4     2.0       1.0  0.0        1.0        1.0        2.0        0.0   
 ..    ...       ...  ...        ...        ...        ...        ...   
 647   1.0       1.0  1.0        1.0        1.0        6.0        1.0   
 648   2.0       1.0  1.0        1.0        1.0        3.0        0.0   
 649   2.0       0.0  0.0        1.0        1.0        1.0        0.0   
 650   2.0       1.0  1.0        1.0        1.0        6.0        0.0   
 651   1.0       1.0  1.0        1.0        0.0        NaN        NaN   
 
      mue_pro_7  mue_pro_10  mue_pro_13  ...  tdvgtotfun_f5viv  \
 0          2.0         2.0         2.0  ...          0.

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base1. 

In [ ]:
fun.transformacion_log( dfs_1_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base2. 

In [ ]:
fun.transformacion_log( dfs_2_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base3. 

In [ ]:
fun.transformacion_log( dfs_3_i )

#### 2.7. Exportar las bases de datos

In [42]:
# Base 0

dfs_0_i[0].to_csv( r'..\..\output\data_preprocess\dfs_0_i_ci.csv', index = False )
dfs_0_i[1].to_csv( r'..\..\output\data_preprocess\dfs_0_i_ca.csv', index = False )
dfs_0_i[2].to_csv( r'..\..\output\data_preprocess\dfs_0_i_mci.csv', index = False )
dfs_0_i[3].to_csv( r'..\..\output\data_preprocess\dfs_0_i_mca.csv', index = False )
dfs_0_i[4].to_csv( r'..\..\output\data_preprocess\dfs_0_i_pci.csv', index = False )
dfs_0_i[5].to_csv( r'..\..\output\data_preprocess\dfs_0_i_pca.csv', index = False )

## 3. Segunda forma: imputación con media y moda

Se reestablece los valores originales de "dfs"

In [ ]:
dfs = [ pd.DataFrame() for x in range( 4 ) ]
for base in bases:
    dfs = bases.copy()

In [ ]:
dfs[0].name = 'base0'
dfs[1].name = 'base1'
dfs[2].name = 'base2'
dfs[3].name = 'base3'

#### 3.1. Filtrar por valores perdidos

Se descartan las variables con un porcentaje de valores perdidos mayor o igual al umbral 0.5

In [ ]:
fun.filtro_missings( dfs, 0.3 )

In [ ]:
fun.determinar_dimensiones ( dfs )

#### 3.2. Imputación

Se imputa con media a las variables de SIAF y con moda a las variables de Renamu. No se generan variables dummy de control.

In [ ]:
fun.imputar_ii( dfs, siaf = True, dummy = False )
fun.imputar_ii( dfs, siaf = False, dummy = False )

In [ ]:
fun.determinar_dimensiones ( dfs )

#### 3.3. Filtro por variabilidad

Se descartan aquellas variables constantes, es decir, con una variabilidad de 0

In [ ]:
fun.filtro_variabilidad( dfs, 0 )

#### 2.4. Filtro por correlación

Se realiza un filtro de correlación para cada una de las variables dependientes de la base0. El umbral es 0.8. Para mayor detalle en el funcionamiento del filtro, véase su respectiva sección en el módulo "funciones.py". Posteriormente se agrupa las bases filtradas de la base 0 en la lista "dfs_0_i". Luego, se asigna el nombre a cada una de las bases. Finalmente, se determina las dimensiones de las bases

In [ ]:
dfs_0_ci = fun.filtro_correlacion( dfs[0], dfs[0][ 'corrup_intensa' ], 0.8 )
dfs_0_ca = fun.filtro_correlacion( dfs[0], dfs[0][ 'corrup_amplia' ], 0.8 )
dfs_0_m = fun.filtro_correlacion( dfs[0], dfs[0][ 'monto' ], 0.8 )

In [ ]:
filtradas_0 = [ dfs_0_ci, dfs_0_ca, dfs_0_m ]
dfs_0_i = []
for i, filtrada_0 in enumerate( filtradas_0 ):
    dfs_0_i.append( filtrada_0 )

In [ ]:
dfs_0_i[0].name = 'dfs_0_i_ci'
dfs_0_i[1].name = 'dfs_0_i_ca'
dfs_0_i[2].name = 'dfs_0_i_m'

In [ ]:
fun.determinar_dimensiones( dfs_0_i )

Se realiza el mismo procedimiento para cada una de las variables dependientes de la base1.

In [ ]:
dfs_1_ci = fun.filtro_correlacion( dfs[1], dfs[1][ 'corrup_intensa' ], 0.8 )
dfs_1_ca = fun.filtro_correlacion( dfs[1], dfs[1][ 'corrup_amplia' ], 0.8 )
dfs_1_m = fun.filtro_correlacion( dfs[1], dfs[1][ 'monto' ], 0.8 )

In [ ]:
filtradas_1 = [ dfs_1_ci, dfs_1_ca, dfs_1_m ]
dfs_1_i = []
for i, filtrada_1 in enumerate( filtradas_1 ):
    dfs_1_i.append( filtrada_1 )

In [ ]:
dfs_1_i[0].name = 'dfs_1_i_ci'
dfs_1_i[1].name = 'dfs_1_i_ca'
dfs_1_i[2].name = 'dfs_1_i_m'

In [ ]:
fun.determinar_dimensiones( dfs_1_i )

Se realiza el mismo procedimiento para cada una de las bases dependientes de la base2.

In [ ]:
dfs_2_ci = fun.filtro_correlacion( dfs[2], dfs[2][ 'corrup_intensa' ], 0.8 )
dfs_2_ca = fun.filtro_correlacion( dfs[2], dfs[2][ 'corrup_amplia' ], 0.8 )
dfs_2_m = fun.filtro_correlacion( dfs[2], dfs[2][ 'monto' ], 0.8 )

In [ ]:
filtradas_2 = [ dfs_2_ci, dfs_2_ca, dfs_2_m ]
dfs_2_i = []
for i, filtrada_2 in enumerate( filtradas_2 ):
    dfs_2_i.append( filtrada_2 )

In [ ]:
dfs_2_i[0].name = 'dfs_2_i_ci'
dfs_2_i[1].name = 'dfs_2_i_ca'
dfs_2_i[2].name = 'dfs_2_i_m'

Se realiza el mismo procedimiento para cada una de las bases dependientes de la base3.

In [ ]:
dfs_3_ci = fun.filtro_correlacion( dfs[3], dfs[3][ 'corrup_intensa' ], 0.8 )
dfs_3_ca = fun.filtro_correlacion( dfs[3], dfs[3][ 'corrup_amplia' ], 0.8 )
dfs_3_m = fun.filtro_correlacion( dfs[3], dfs[3][ 'monto' ], 0.8 )

In [ ]:
filtradas_3 = [ dfs_3_ci, dfs_3_ca, dfs_3_m ]
dfs_3_i = []
for i, filtrada_3 in enumerate( filtradas_3 ):
    dfs_3_i.append( filtrada_3 )

In [ ]:
dfs_3_i[0].name = 'dfs_3_i_ci'
dfs_3_i[1].name = 'dfs_3_i_ca'
dfs_3_i[2].name = 'dfs_3_i_m'

#### 2.5. Imputar outliers

Se imputa los valores del percentil 1% superior con los valores del percentil 99% para cada una de las variables de SIAF. En este caso, la imputación se realiza sobre las tres bases resultantes de la base0.

In [ ]:
fun.imputar_outliers( dfs_0_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base1.

In [ ]:
fun.imputar_outliers( dfs_1_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base2.

In [ ]:
fun.imputar_outliers( dfs_2_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base3.

In [ ]:
fun.imputar_outliers( dfs_3_i, vn.siaf_variables, 0.01 )

#### 3.6. Transformaciones logarítmicas

Se realiza una transformación logarítmica a todas las variables pertenecientes a SIAF. La transformación se aplica sobre las tres bases de datos resultantes de la base0.

In [ ]:
fun.transformacion_log( dfs_0_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base1. 

In [ ]:
fun.transformacion_log( dfs_1_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base2. 

In [ ]:
fun.transformacion_log( dfs_2_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base3. 

In [ ]:
fun.transformacion_log( dfs_3_i )

## 4. Tercera forma: imputación con deep learning mediante la librería datawig

Se reestablece los valores originales de "dfs"

In [ ]:
dfs = [ pd.DataFrame() for x in range( 4 ) ]
for base in bases:
    dfs = bases.copy()

In [ ]:
dfs[0].name = 'base0'
dfs[1].name = 'base1'
dfs[2].name = 'base2'
dfs[3].name = 'base3'

#### 4.1. Filtrar por valores perdidos

Se descartan las variables con un porcentaje de valores perdidos mayor o igual al umbral 0.5

In [ ]:
fun.filtro_missings( dfs, 0.5 )

In [ ]:
fun.determinar_dimensiones ( dfs )

#### 4.2. Imputación

Se imputa usando deep learning mediante el algoritmo datawig. No se generan variables dummy de control. Para mayor detalle, sobre la forma de imputación, revísese la función "imputar_iii" en el módulo "funciones.py". Se recomienda usar este método de imputación con precaución. De momento, la función "Imputar_iii" está comentada en el módulo. En caso se desee usar, debe descomentar en el módulo "funciones.py" la función, los paquetes vinculados a datawig, y crear un envirment con la versión de python 3.7.

In [ ]:
dfs[0] = fun.imputar_iii( dfs[0] )
dfs[1] = fun.imputar_iii( dfs[1] )
dfs[2] = fun.imputar_iii( dfs[2] )
dfs[3] = fun.imputar_iii( dfs[3] )

In [ ]:
fun.determinar_dimensiones ( dfs )

#### 4.3. Filtro por variabilidad

Se descartan aquellas variables constantes, es decir, con una variabilidad de 0

In [ ]:
fun.filtro_variabilidad( dfs, 0 )

#### 4.4. Filtro por correlación

Se realiza un filtro de correlación para cada una de las variables dependientes de la base0. El umbral es 0.8. Para mayor detalle en el funcionamiento del filtro, véase su respectiva sección en el módulo "funciones.py". Posteriormente se agrupa las bases filtradas de la base 0 en la lista "dfs_0_i". Luego, se asigna el nombre a cada una de las bases. Finalmente, se determina las dimensiones de las bases

In [ ]:
dfs_0_ci = fun.filtro_correlacion( dfs[0], dfs[0][ 'corrup_intensa' ], 0.8 )
dfs_0_ca = fun.filtro_correlacion( dfs[0], dfs[0][ 'corrup_amplia' ], 0.8 )
dfs_0_m = fun.filtro_correlacion( dfs[0], dfs[0][ 'monto' ], 0.8 )

In [ ]:
filtradas_0 = [ dfs_0_ci, dfs_0_ca, dfs_0_m ]
dfs_0_i = []
for i, filtrada_0 in enumerate( filtradas_0 ):
    dfs_0_i.append( filtrada_0 )

In [ ]:
dfs_0_i[0].name = 'dfs_0_i_ci'
dfs_0_i[1].name = 'dfs_0_i_ca'
dfs_0_i[2].name = 'dfs_0_i_m'

In [ ]:
fun.determinar_dimensiones( dfs_0_i )

Se realiza el mismo procedimiento para cada una de las variables dependientes de la base1.

In [ ]:
dfs_1_ci = fun.filtro_correlacion( dfs[1], dfs[1][ 'corrup_intensa' ], 0.8 )
dfs_1_ca = fun.filtro_correlacion( dfs[1], dfs[1][ 'corrup_amplia' ], 0.8 )
dfs_1_m = fun.filtro_correlacion( dfs[1], dfs[1][ 'monto' ], 0.8 )

In [ ]:
filtradas_1 = [ dfs_1_ci, dfs_1_ca, dfs_1_m ]
dfs_1_i = []
for i, filtrada_1 in enumerate( filtradas_1 ):
    dfs_1_i.append( filtrada_1 )

In [ ]:
dfs_1_i[0].name = 'dfs_1_i_ci'
dfs_1_i[1].name = 'dfs_1_i_ca'
dfs_1_i[2].name = 'dfs_1_i_m'

In [ ]:
fun.determinar_dimensiones( dfs_1_i )

Se realiza el mismo procedimiento para cada una de las bases dependientes de la base2.

In [ ]:
dfs_2_ci = fun.filtro_correlacion( dfs[2], dfs[2][ 'corrup_intensa' ], 0.8 )
dfs_2_ca = fun.filtro_correlacion( dfs[2], dfs[2][ 'corrup_amplia' ], 0.8 )
dfs_2_m = fun.filtro_correlacion( dfs[2], dfs[2][ 'monto' ], 0.8 )

In [ ]:
filtradas_2 = [ dfs_2_ci, dfs_2_ca, dfs_2_m ]
dfs_2_i = []
for i, filtrada_2 in enumerate( filtradas_2 ):
    dfs_2_i.append( filtrada_2 )

In [ ]:
dfs_2_i[0].name = 'dfs_2_i_ci'
dfs_2_i[1].name = 'dfs_2_i_ca'
dfs_2_i[2].name = 'dfs_2_i_m'

Se realiza el mismo procedimiento para cada una de las bases dependientes de la base3.

In [ ]:
dfs_3_ci = fun.filtro_correlacion( dfs[3], dfs[3][ 'corrup_intensa' ], 0.8 )
dfs_3_ca = fun.filtro_correlacion( dfs[3], dfs[3][ 'corrup_amplia' ], 0.8 )
dfs_3_m = fun.filtro_correlacion( dfs[3], dfs[3][ 'monto' ], 0.8 )

In [ ]:
filtradas_3 = [ dfs_3_ci, dfs_3_ca, dfs_3_m ]
dfs_3_i = []
for i, filtrada_3 in enumerate( filtradas_3 ):
    dfs_3_i.append( filtrada_3 )

In [ ]:
dfs_3_i[0].name = 'dfs_3_i_ci'
dfs_3_i[1].name = 'dfs_3_i_ca'
dfs_3_i[2].name = 'dfs_3_i_m'

#### 4.5. Imputar outliers

Se imputa los valores del percentil 1% superior con los valores del percentil 99% para cada una de las variables de SIAF. En este caso, la imputación se realiza sobre las tres bases resultantes de la base0.

In [ ]:
fun.imputar_outliers( dfs_0_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base1.

In [ ]:
fun.imputar_outliers( dfs_1_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base2.

In [ ]:
fun.imputar_outliers( dfs_2_i, vn.siaf_variables, 0.01 )

Se aplica el mismo procedimiento para las tres bases resultantes de la base3.

In [ ]:
fun.imputar_outliers( dfs_3_i, vn.siaf_variables, 0.01 )

#### 4.6. Transformaciones logarítmicas

Se realiza una transformación logarítmica a todas las variables pertenecientes a SIAF. La transformación se aplica sobre las tres bases de datos resultantes de la base0.

In [ ]:
fun.transformacion_log( dfs_0_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base1. 

In [ ]:
fun.transformacion_log( dfs_1_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base2. 

In [ ]:
fun.transformacion_log( dfs_2_i )

In [ ]:
Se aplica el mismo procedimiento para las tres bases de datos resultantes de la base3. 

In [ ]:
fun.transformacion_log( dfs_3_i )